# Kelvin-Helmholtz Instability with SPH (Continued)

By now you should have run a simulation and got a reasonable number of snapshots in the output. This notebook will now ingest those outputs and 

## What we'll do in this notebook

1. **Visualize the results** and create movies
2. **Explore different solvers/parameters** to see how they affect the instability growth

Let's get started!

In [1]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
import unyt
from swiftsimio import load

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. Analysis and Visualization

Let's create some analysis functions that work whether or not you've run the simulation:

In [2]:
def plot_gas_snapshot(filename, title="", save_path=None):
    """
    Load a SWIFT snapshot (gas only) with swiftsimio and make simple scatter plots:
      - Density-coloured positions if density exists, else plain positions
      - X-velocity
      - Velocity magnitude
    """
    ds = load(filename)
    gas = ds.gas

    # Convert to plain numpy in common units
    pos = gas.coordinates.to("kpc").value                 # (N, 3)
    vel = gas.velocities.to("km/s").value                 # (N, 3)
    dens = gas.density.to("g/cm**3").value if hasattr(gas, "density") else None
    time_Gyr = float(ds.metadata.time.to("Gyr"))

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    # Panel 1: density (or fallback to positions)
    if dens is not None:
        im1 = axes[0].scatter(pos[:, 0], pos[:, 1],
                              c=np.log10(dens + 1e-99), s=2, cmap="viridis")
        axes[0].set_title(f"log10 Density (t={time_Gyr:.3f} Gyr)")
        plt.colorbar(im1, ax=axes[0], label="log10(g cm$^{-3}$)")
    else:
        axes[0].scatter(pos[:, 0], pos[:, 1], s=1, alpha=0.6)
        axes[0].set_title(f"Gas positions (t={time_Gyr:.3f} Gyr)")

    axes[0].set_xlabel("x [kpc]")
    axes[0].set_ylabel("y [kpc]")
    axes[0].set_aspect("equal")

    # Panel 2: x-velocity
    im2 = axes[1].scatter(pos[:, 0], pos[:, 1], c=vel[:, 0], s=2, cmap="RdBu_r")
    axes[1].set_title("X-Velocity")
    axes[1].set_xlabel("x [kpc]")
    axes[1].set_ylabel("y [kpc]")
    axes[1].set_aspect("equal")
    plt.colorbar(im2, ax=axes[1], label="km s$^{-1}$")

    # Panel 3: velocity magnitude (2D plane)
    vel_mag = np.sqrt(vel[:, 0]**2 + vel[:, 1]**2)
    im3 = axes[2].scatter(pos[:, 0], pos[:, 1], c=vel_mag, s=2, cmap="plasma")
    axes[2].set_title("Velocity Magnitude")
    axes[2].set_xlabel("x [kpc]")
    axes[2].set_ylabel("y [kpc]")
    axes[2].set_aspect("equal")
    plt.colorbar(im3, ax=axes[2], label="km s$^{-1}$")

    if title:
        fig.suptitle(title, fontsize=16)
    plt.tight_layout()

    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches="tight")
    plt.show()


## 7. Loading and Analyzing Simulation Results

If the simulation ran successfully, let's load and analyze the results:

In [ ]:
import glob

# Look for snapshot files
run_name = "GADGET2"
snapshot_files = sorted(glob.glob(f'../runs/{run_name}/kelvin_helmholtz_*.hdf5'))

if snapshot_files:
    print(f"Found {len(snapshot_files)} snapshot files")
    
    # Load and plot a few snapshots
    snapshot_indices = [0, len(snapshot_files)//4, len(snapshot_files)//2, -1]
    
    for i in snapshot_indices:
        if i < len(snapshot_files):
            print(f"\nLoading snapshot {i}: {snapshot_files[i]}")
            snapshot_data = load_swift_snapshot(snapshot_files[i])
            if snapshot_data:
                plot_snapshot(snapshot_data, 
                             f"Kelvin-Helmholtz Instability Evolution")
else:
    print("No snapshot files found. Is the path correct?")

No snapshot files found. Is the path correct?


## Summary and Next Steps

In this notebook, we've:

1. **Learned about the Kelvin-Helmholtz instability** - its physical origin and importance
2. **Generated initial conditions** with two fluid layers and velocity shear
3. **Created SWIFT-compatible files** for running SPH simulations
4. **Set up parameter files** for the simulation
5. **Developed analysis and visualization tools**
6. **Explored parameter variations** and their effects

### Key Physics Insights:
- The Kelvin-Helmholtz instability grows from small perturbations in shear flows
- Density contrasts can stabilize or destabilize the interface
- The characteristic "cat's eye" structures develop from the velocity shear

### Next Steps:
1. **Run the simulation** with SWIFT if you have it installed
2. **Experiment with different parameters** - try higher density contrasts, different perturbation wavelengths
3. **Add physical effects** - viscosity, cooling, magnetic fields
4. **Move to 3D** - the instability has rich 3D structure

### Challenges to Try:
- Implement multi-wavelength perturbations
- Add a background flow
- Try different density profiles (gradual vs sharp interfaces)
- Compare with linear stability theory predictions

The Kelvin-Helmholtz instability is fundamental to many astrophysical phenomena, from planetary atmospheres to accretion disk flows. Understanding its behavior in SPH simulations is crucial for modeling real astrophysical systems!